In [ ]:
import pandas as pd
import numpy as np

# Visualization
from matplotlib import pyplot as plt
import seaborn as sns
import itertools

### Plot Mid Scores over Assignment

In [ ]:
mid_scores = pd.read_csv('./data/mid_scores.csv')
mid_scores.sort_values('AssignmentID')

In [ ]:
sns.scatterplot(data=mid_scores, x='AssignmentID', y='MID_Score', hue='SubjectID')

In [ ]:
sns.scatterplot(data=mid_scores, x='MID_Score', y='AssignmentScore')
plt.savefig('./images/AssignmentScore_MidScore.png')

In [ ]:
mid_scores.columns

In [ ]:
variables = filter(lambda v: v != 'Unnamed: 0', mid_scores.columns)
pairs = list(itertools.combinations(variables, 2))

fig, axs = plt.subplots(len(pairs) // 2, 2, figsize=(12,48))
for col in range(len(pairs) // 2):
    for row in range(2):
        pair = pairs.pop(0)
        ax = axs[col][row]
        sns.scatterplot(data=mid_scores, x=pair[0], y=pair[1], hue='Incremental', ax=ax)
        ax.set_title(f'{pair[0]} vs {pair[1]}')
fig.tight_layout()

In [ ]:
sns.boxplot(data=mid_scores, x='AssignmentID', y='MID_Score')

In [ ]:
sns.countplot(data=mid_scores, x='AssignmentID')